In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    SELECT * from
        (SELECT count(crim) as crim FROM boston where boston.crim is null),
        (SELECT count(zn) as zn FROM boston where boston.zn is null),
        (SELECT count(indus) as indus FROM boston where boston.indus is null),
        (SELECT count(chas) as chas FROM boston where boston.chas is null),
        (SELECT count(nox) as nox FROM boston where boston.nox is null)
    """,
    con
)

,crim,zn,indus,chas,nox
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    """
    SELECT count(distinct crim) as crim,
        count(distinct zn) as zn,
        count(distinct indus) as indus,
        count(distinct chas) as chas,
        count(distinct nox) as nox
    FROM boston

    """,
    con,
)

,crim,zn,indus,chas,nox
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
pd.read_sql(
    """
    SELECT Column_Name, Diff FROM (
        SELECT 'CRIM' as Column_Name, max(CRIM) - min(CRIM) as Diff FROM (
            SELECT CRIM,
            NTILE(2) OVER(ORDER BY CRIM) AS 'NTILE_CRIM'
            FROM boston
            )
        WHERE NTILE_CRIM=1
        UNION
        SELECT 'ZN', max(ZN) - min(ZN) as Diff FROM (
            SELECT ZN,
            NTILE(2) OVER(ORDER BY ZN) AS 'NTILE_ZN'
            FROM boston
            )
        WHERE NTILE_ZN=1
        UNION
        SELECT 'INDUS', max(INDUS) - min(INDUS) as Diff FROM (
            SELECT INDUS,
            NTILE(2) OVER(ORDER BY INDUS) AS 'NTILE_INDUS'
            FROM boston
            )
        WHERE NTILE_INDUS=1
        UNION
        SELECT 'CHAS', max(CHAS) - min(CHAS) as Diff FROM (
            SELECT CHAS,
            NTILE(2) OVER(ORDER BY CHAS) AS 'NTILE_CHAS'
            FROM boston
            )
        WHERE NTILE_CHAS=1
        UNION
        SELECT 'NOX', max(NOX) - min(NOX) as Diff FROM (
            SELECT NOX,
            NTILE(2) OVER(ORDER BY NOX) AS 'NTILE_NOX'
            FROM boston
            )
        WHERE NTILE_NOX=1)
    WHERE Diff = 0
    """,
    con,
)

# Вывод: Медиана равна минимальному значению

,Column_Name,Diff
0,CHAS,0.0
1,ZN,0.0


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
pd.read_sql(
    """
    WITH 
        high_price AS 
            (SELECT avg(RM) as highrest_price FROM (
                SELECT RM, MEDV, max(MEDV) over() max_MEDV
                FROM boston)
            WHERE MEDV = max_MEDV)
    SELECT * FROM (SELECT 
            25 AS Houses_n,
            avg(RM) AS Avg_RM,
            highrest_price AS Avg_RM_highrest_price,
            highrest_price - avg(RM) AS Diff
        FROM
            (SELECT RM, MEDV
            FROM boston
            ORDER BY MEDV ASC LIMIT 25), high_price
    )
    UNION
    SELECT * FROM (SELECT 
            50 AS Houses_n,
            avg(RM) AS Avg_RM,
            highrest_price AS Avg_RM_highrest_price,
            highrest_price-avg(RM) AS Diff
        FROM
            (SELECT RM, MEDV
            FROM boston
            ORDER BY MEDV ASC LIMIT 50), high_price)
    UNION
    SELECT * FROM (SELECT 
            100 AS Houses_n,
            avg(RM) AS Avg_RM,
            highrest_price AS Avg_RM_highrest_price,
            highrest_price-avg(RM) AS Diff
        FROM
            (SELECT RM, MEDV
            FROM boston
            ORDER BY MEDV ASC LIMIT 100), high_price)
    UNION
    SELECT * FROM (SELECT 
            200 AS Houses_n,
            avg(RM) AS Avg_RM,
            highrest_price AS Avg_RM_highrest_price,
            highrest_price-avg(RM) AS Diff 
        FROM
            (SELECT RM, MEDV
            FROM boston
            ORDER BY MEDV ASC LIMIT 200), high_price)
    UNION
    SELECT * FROM (SELECT 
            300 AS Houses_n,
            avg(RM) AS Avg_RM,
            highrest_price AS Avg_RM_highrest_price,
            highrest_price-avg(RM) AS Diff
        FROM
            (SELECT RM, MEDV
            FROM boston
            ORDER BY MEDV ASC LIMIT 300), high_price)
    """,
    con,
)

#Влияние есть, по мере увеличения выборки уменьшается разница в количестве комнат

,Houses_n,Avg_RM,Avg_RM_highrest_price,Diff
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [11]:
pd.read_sql(
    """
    SELECT Rank FROM (SELECT
        LSTAT, MEDV, RANK() OVER(ORDER BY LSTAT ASC) as Rank FROM boston     
    )
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    """,
    con,
)
#Высокая стоимость домов означает малое количество населения с более низким статусом.

,Rank
0,1
1,2
2,6
3,8
4,9
5,15
6,17
7,18
8,26
9,27


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
pd.read_sql(
    """
    SELECT avg(MEDV) as avg_MEDV, CHAS
    FROM boston
    group by CHAS
    """,
    con,  
)

,avg_MEDV,CHAS
0,22.093843,0.0
1,28.440000,1.0


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
pd.read_sql(
    """
    SELECT Column_Name, Boolean FROM 
    (SELECT 
        'CRIM' AS Column_Name, 
        (SELECT avg(CRIM) FROM boston WHERE CHAS = 1) > avg(crim) AS Boolean
    FROM boston
    UNION
    SELECT 
        'ZN' AS Column_Name, 
        (SELECT avg(ZN) FROM boston WHERE CHAS = 1) > avg(ZN) AS Boolean
    FROM boston
    UNION
    SELECT 
        'INDUS' AS Column_Name, 
        (SELECT avg(INDUS) FROM boston WHERE CHAS = 1) > avg(INDUS) AS Boolean
    FROM boston
    UNION
    SELECT 
        'NOX' AS Column_Name, 
        (SELECT avg(NOX) FROM boston WHERE CHAS = 1) > avg(NOX) AS Boolean
    FROM boston
    UNION
    SELECT 
        'RM' AS Column_Name, 
        (SELECT avg(RM) FROM boston WHERE CHAS = 1) > avg(RM) AS Boolean
    FROM boston
    UNION
    SELECT 
        'DIS' AS Column_Name, 
        (SELECT avg(DIS) FROM boston WHERE CHAS = 1) > avg(DIS) AS Boolean
    FROM boston
    UNION
    SELECT 
        'RAD' AS Column_Name, 
        (SELECT avg(RAD) FROM boston WHERE CHAS = 1) > avg(RAD) AS Boolean
    FROM boston
    UNION
    SELECT 
        'TAX' AS Column_Name, 
        (SELECT avg(TAX) FROM boston WHERE CHAS = 1) > avg(TAX) AS Boolean
    FROM boston
    UNION
    SELECT 
        'PTRATIO' AS Column_Name, 
        (SELECT avg(PTRATIO) FROM boston WHERE CHAS = 1) > avg(PTRATIO) AS Boolean
    FROM boston
    UNION
    SELECT 
        'B' AS Column_Name, 
        (SELECT avg(B) FROM boston WHERE CHAS = 1) > avg(B) AS Boolean
    FROM boston
    UNION
    SELECT 
        'LSTAT' AS Column_Name, 
        (SELECT avg(LSTAT) FROM boston WHERE CHAS = 1) > avg(LSTAT) AS Boolean
    FROM boston
    )
    WHERE Boolean = 1
    """,
    con,
)

#У рек большая концентрация пром зон => более высокий уровень оксидов азота.
#Судя по предыдущему заданию, цена на недвижимость у рек выше => обычно там более этиная недвижимость
#=> среднее количество комнот выше

,Column_Name,Boolean
0,B,1
1,INDUS,1
2,NOX,1
3,RM,1


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [14]:
pd.read_sql(
    """
    WITH
    INDUS_percentile as (SELECT INDUS, NTILE(10) OVER (ORDER BY INDUS) AS Percentile
              FROM boston),
    NOX_percentile as (SELECT NOX, NTILE(10) OVER (ORDER BY NOX) AS Percentile
              FROM boston)
    SELECT INDUS, NOX, INDUS_percentile.Percentile FROM INDUS_percentile, NOX_percentile
    WHERE INDUS_percentile.Percentile = NOX_percentile.Percentile
    GROUP BY INDUS_percentile.Percentile
    """,
    con,
)

#Наблюдается корреляция

,INDUS,NOX,Percentile
0,0.46,0.3850,1
1,2.93,0.4280,2
2,4.39,0.4429,3
3,5.96,0.4720,4
4,7.38,0.5070,5
5,9.69,0.5380,6
6,13.89,0.5800,7
7,18.10,0.6090,8
8,18.10,0.6710,9
9,19.58,0.7130,10
